In [1]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!python -m spacy download ru_core_news_sm

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     --------------------------------------- 15.3/15.3 MB 10.4 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [3]:
import spacy
nlp = spacy.load("ru_core_news_sm")

In [4]:
import sqlite3

In [106]:
class DBHandler:
    conn = None
    cur = None

    def __init__(self, db_path):
        print('init')
        self.conn = sqlite3.connect(db_path, check_same_thread=False)
        self.cur = self.conn.cursor()

    def __del__(self):
        self.conn.close()

    def add_text(self, text_name):
        
        self.cur.execute('''INSERT or IGNORE INTO texts (name)
                            VALUES (?)
                            RETURNING id''', (text_name, ))
    
        added_id = self.cur.fetchone()
        self.conn.commit()
        
        if not added_id:
            self.cur.execute('''SELECT id
                                FROM texts
                                WHERE name = ?''', (text_name, ))
            added_id = self.cur.fetchone()
        return added_id[0]
        
    def add_sent(self, text_id, sent, lemmatized):
        
        self.cur.execute('''INSERT INTO sents (text_id, sent, lemmatized)
                            VALUES (?, ?, ?)
                            RETURNING id''', (text_id, sent, lemmatized))
        
        added_id = self.cur.fetchone()
        self.conn.commit()
        return added_id[0]
    
    def add_tokens(self, tokens_info): 
        ":param tokens_info: list of tuples (sent_id, word_in_sent, token)"
        
        self.cur.executemany('''INSERT INTO tokens (sent_id, word_in_sent, token)
                                VALUES (?, ?, ?)''', tokens_info)
        
        self.conn.commit()
        
        self.cur.execute('''SELECT id 
                            FROM tokens
                            WHERE sent_id = ?
                            ORDER BY word_in_sent''', (tokens_info[0][0],))
        added_ids = self.cur.fetchall()
        return [token_id[0] for token_id in added_ids]

    
#     def get_lemma_id(self)
    
    def add_lemmas(self, lemmas):
        self.cur.executemany('''INSERT or IGNORE
                            INTO lemma (name)
                            VALUES (?)''', lemmas)
        self.cur.commit()
        
        self.cur.execute('''SELECT id
                            FROM lemma
                            WHERE name IN ?''', lemmas)
        
        lemma_ids = self.cur.fetchall()
        return [token_id[0] for token_id in lemma_ids]
db = DBHandler('math_corpus_new.db')

init


In [37]:
xml_filename = 'test.xml'

In [7]:
from bs4 import BeautifulSoup

In [11]:
with open(xml_filename, 'r', encoding='utf-8') as f:
    xml_file = f.read()

In [12]:
bs_data = BeautifulSoup(xml_file, "xml")

In [13]:
segs = bs_data.find_all('segment')
segs

[<segment features="math_tags" filename="" id="1" state="active" subcorp=""><segment comment="Модель управления: X в Y.ORD.LOC.SG степени; X в степени Y.NOM.SG" features="math_tags;oper;power" filename="" id="110" parent="1" state="active" subcorp="">Степень</segment> с <segment features="math_tags;term;set;nat" filename="" id="113" parent="1" state="active" subcorp="">натуральным</segment> <segment features="math_tags;visual;superscript" filename="" id="114" parent="1" state="active" subcorp="">показателем</segment>.</segment>,
 <segment comment="Модель управления: X в Y.ORD.LOC.SG степени; X в степени Y.NOM.SG" features="math_tags;oper;power" filename="" id="110" parent="1" state="active" subcorp="">Степень</segment>,
 <segment features="math_tags;term;set;nat" filename="" id="113" parent="1" state="active" subcorp="">натуральным</segment>,
 <segment features="math_tags;visual;superscript" filename="" id="114" parent="1" state="active" subcorp="">показателем</segment>,
 <segment feat

In [88]:
db.add_text('Степень с натуральным показателем.')

3

In [19]:
t = 'Определение степени: а в энной степени равняется произведению энн множителей, каждый из которых равен а, где энн - число натуральное. Считается, что а в первой степени равняется а.'

In [28]:
sent = list(nlp(t).sents)[0]

In [52]:
list(sent)

[Например,
 :,
 два,
 в,
 пятой,
 степени,
 разделим,
 на,
 два,
 в,
 третьей,
 степени,
 ,,
 вычитаем,
 показатели,
 ,,
 получаем,
 два,
 во,
 второй,
 степени,
 и,
 четыре,
 .]

In [107]:
lemmas = ['вычитать', 'степень', 'два']
db.add_lemmas(lemmas)

OperationalError: near "?": syntax error

In [82]:
word_counts = tuple(range(len(sent)))
tokens = list(sent)
sent_id = 3
# for t in sent:
#     print(t)
print(tokens[0].text)
to_write = [(sent_id, i, tokens[i].text) for i in word_counts]

Например


In [83]:
to_write

[(3, 0, 'Например'),
 (3, 1, ':'),
 (3, 2, 'два'),
 (3, 3, 'в'),
 (3, 4, 'пятой'),
 (3, 5, 'степени'),
 (3, 6, 'разделим'),
 (3, 7, 'на'),
 (3, 8, 'два'),
 (3, 9, 'в'),
 (3, 10, 'третьей'),
 (3, 11, 'степени'),
 (3, 12, ','),
 (3, 13, 'вычитаем'),
 (3, 14, 'показатели'),
 (3, 15, ','),
 (3, 16, 'получаем'),
 (3, 17, 'два'),
 (3, 18, 'во'),
 (3, 19, 'второй'),
 (3, 20, 'степени'),
 (3, 21, 'и'),
 (3, 22, 'четыре'),
 (3, 23, '.')]

In [99]:
ids = db.add_tokens(to_write)

3


In [100]:
ids

[97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120]

In [49]:
for seg in segs:
    parent_id = seg.get('parent')
    if not parent_id:
        text = seg.get_text()
        sentences = list(nlp(text).sents)
        print(sentences)
        for sent in sentences:
            word_counts = tuple(range(len(sent)))
            tokens = list(sent)
            lemmas = [t.lemma_ for t in sent]
            lemmatized = ' '.join(lemmas)
            sent_id = db.add_sent(text_id, sent.text, lemmatized)
            tokens_to_write = [(sent_id, i, tokens[i].text) for i in word_counts]
            db.add_tokens(tokens_to_write)
            pass
    else:
        print(seg.get_text(), seg.get('features'))

[Степень с натуральным показателем.]
Степень с натуральным показателем.
Степень math_tags;oper;power
натуральным math_tags;term;set;nat
показателем math_tags;visual;superscript
[Определение степени: а в энной степени равняется произведению энн множителей, каждый из которых равен а, где энн - число натуральное., Считается, что а в первой степени равняется а.]
Определение степени: а в энной степени равняется произведению энн множителей, каждый из которых равен а, где энн - число натуральное.
Считается, что а в первой степени равняется а.
Определение math_tags;comment;definition
степени math_tags;oper;power
а math_tags;term;var;common;letter;latin;a
энной math_tags;term;var;common;letter;latin;n
степени math_tags;oper;power
равняется math_tags;pred;equal
произведению math_tags;oper;mult;common;self
энн math_tags;term;var;common;letter;latin;n
множителей math_tags;term
равен math_tags;pred;equal;common;self
а math_tags;term;var;common;letter;latin;a
энн math_tags;term;var;common;letter;lat